##### Install all the required configuration

In [1]:
from llama_cpp import Llama
from langchain import PromptTemplate
import random
import re
import ast
import gradio as gr
import logging


###### The main purpose of the gadio:
<ul>
<li>Presenting or for client to experient with</li>
<ul>
<li> Changes wrt code make <b>demo.launch</b> make <b>share=True</b></li>
</ul>
<li>For <b>prompt optimaztion</b></li>
<li><b>Fine-tuning</b> model parameters</li>
</ul>

Logging into appropritate file to check logs

In [2]:
logging.basicConfig(filename='example.log', filemode='w+', format="%(asctime)s :[%(levelname)s]: %(message)s", level=logging.INFO)
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

Update the model path wrt your local

<ul>
<li>Huggingface model:<a href="https://huggingface.co/TheBloke/Llama-2-7B-GGML">model link</a></li>
<li>Checkout <b>Provided file</b></li>
<li>Go to file versions</li>
<li>Download the required model</li>
<li><b>llama-2-7b.ggmlv3.q8_0.bin</b> is the largest which is working as expected</li>
</ul>


In [3]:
model_id = 'llama-2-7b.ggmlv3.q8_0.bin'


In [4]:
# working well feel free to experiment with
seed = 1296

##### n_ctx
<ul>
<li>n_ctx is Maximum context size</li>
<li>Default value is 512</li>
<li>Please fine-tune as per the input parameters</li>
</ul>

In [5]:
local_llm = Llama(model_path=model_id, n_ctx=1024, seed=random.randint(0,1337))
local_llm

llama.cpp: loading model from /Users/fission/Downloads/LLama-2-compressed/llama-2-7b.ggmlv3.q8_0.bin
llama_model_load_internal: format     = ggjt v3 (latest)
llama_model_load_internal: n_vocab    = 32000
llama_model_load_internal: n_ctx      = 1024
llama_model_load_internal: n_embd     = 4096
llama_model_load_internal: n_mult     = 256
llama_model_load_internal: n_head     = 32
llama_model_load_internal: n_head_kv  = 32
llama_model_load_internal: n_layer    = 32
llama_model_load_internal: n_rot      = 128
llama_model_load_internal: n_gqa      = 1
llama_model_load_internal: rnorm_eps  = 5.0e-06
llama_model_load_internal: n_ff       = 11008
llama_model_load_internal: freq_base  = 10000.0
llama_model_load_internal: freq_scale = 1
llama_model_load_internal: ftype      = 7 (mostly Q8_0)
llama_model_load_internal: model size = 7B
llama_model_load_internal: ggml ctx size =    0.08 MB
llama_model_load_internal: mem required  = 6828.73 MB (+  512.00 MB per state)
llama_new_context_with_model: k

In [6]:
def format_data(data, expected_type):
    """
    converting data to expected format else returning empty
    :param data:
    :param expected_type:
    :return:
    """
    default_type = None
    if expected_type==list:
        default_type = []
    elif expected_type==dict:
        default_type = {}
    elif expected_type==set:
        default_type = set([])
    elif expected_type==bool:
        default_type = False
    try:
        if type(data)==expected_type:
            return data
        if expected_type==bool:
            if "true" in data or "True" in data:
                default_type = True
            return default_type
        if type(data) == str and expected_type in [list, list, tuple, dict]:
            if len(data)>0:
                default_type = ast.literal_eval(data)
    except Exception as ex:
        logger.error(f"Error while formatting data: {ex}")
    return default_type

Validating <b>format_data</b> function which is used for formatting data

In [7]:
format_data("{'Keywords': 'Fire-Boltt Phoenix Smart Watch, 1.3 Inch display, 240 Watts Lithium Ion batteries, Bluetooth Connectivity, Android OS, 5 Days of Battery life, Made in China','Description':''}", dict)

{'Keywords': 'Fire-Boltt Phoenix Smart Watch, 1.3 Inch display, 240 Watts Lithium Ion batteries, Bluetooth Connectivity, Android OS, 5 Days of Battery life, Made in China',
 'Description': ''}

In [8]:
format_data(None, dict)

{}

In [9]:
def get_prompt_cleaned_format(template, data, sep="\n ", replace_words={}):
    """
    generating prompt in text format
    you can only send prompt taking as input as well
    Please modify as per requirements
    :param template: Template format
    :param data: data is in dictionary format
    :param sep: please update it as per the prompt
    :return:
    """
    try:
        if data is not None and type(data)==str and len(data.strip)!=0:
            data = {}
        if "{information}" not in template:
            template = template.strip()
            template += " {information}"
        data = format_data(data, dict)
        prompt = PromptTemplate(
            input_variables=["information"],
            template=template)
        info = ""
        i = 1
        for key,val in data.items():
            if key is not None and val is not None:
                if key is None:
                    key = ""
                if val is None:
                    val = ""
                info += key+": "
                info += val
                if i<len(data):
                    info += sep
            i += 1
        info = info.strip()

        # If you want prompt can only send prompt as per the requirement without formatting
        # please modify the code as per your requirements.
        output =  prompt.format(information=info).strip()
        if len(replace_words)>0:
            for key,val in replace_words.items():
                output = output.replace(key,val)
            output = " ".join(output.split()).strip()
        # print(output)
        return output
    except Exception as ex:
        # print(f"Error while generating prompt: {ex}")
        logger.error(f"Error while generating prompt: {ex}")
        return ""

##### Sample Template for generating Text given the words

In [10]:
template_1 = "Generate a spellbinding Description using only the input Keywords. " \
             "Don't use Price details, HTML tags, Javascript, Programming languages, URLs, or information " \
             "not in the input Keywords to generate the Description.\n {information}"
data_1 = {"Keywords": "Fire-Boltt Phoenix Smart Watch, 1.3 Inch display, 240 Watts Lithium Ion batteries, Bluetooth Connectivity, "
                      "Android OS, 5 Days of Battery life, Made in China",
          "Description":""}

##### Template output

In [11]:
get_prompt_cleaned_format(template_1, data_1)

"Generate a spellbinding Description using only the input Keywords. Don't use Price details, HTML tags, Javascript, Programming languages, URLs, or information not in the input Keywords to generate the Description.\n Keywords: Fire-Boltt Phoenix Smart Watch, 1.3 Inch display, 240 Watts Lithium Ion batteries, Bluetooth Connectivity, Android OS, 5 Days of Battery life, Made in China\n Description:"

#### Handling the regular question answering

In [12]:
template_2 = "Please Answer to the Question in a sentence.\n {information}"
data_2 = {"Question":"What is the Capital of India?", "Answer":""}
get_prompt_cleaned_format(template_2, data_2)

'Please Answer to the Question in a sentence.\n Question: What is the Capital of India?\n Answer:'

##### When the input data is None

In [13]:
template_3 = "Please Answer to the Question in a sentence.\n What is the capital of India? {information}"
data_3 = None
get_prompt_cleaned_format(template_3, data_3)


'Please Answer to the Question in a sentence.\n What is the capital of India?'

##### When the <b>{information}</b> if missing from template

In [14]:
template_4 = "Please Answer to the Question in a sentence.\n What is the capital of India?"
data_4 = None
get_prompt_cleaned_format(template_4, data_4)


'Please Answer to the Question in a sentence.\n What is the capital of India?'

##### Please feel free to experiment with stop words

In [15]:
stop_words = ["keywords","Keywords", "Keywords:", "Keywords:\n", "Specifications:", "Specifications:\n",
              "Requiements:", "Requiements:\n", "Responsibilities", "Responsibilities:", "Responsibilities:\n",
              "Features:", "Features:\n", "http://", "https://", "www.", ".com", ".php", "<", "<head>", "<body>",
              "<header>", "<hr>", "<input>", "<b>", "<a ","href=", "<ul>", "<li>","*", "* ", "<html>", "</b>", "</a>",
              "</ul>","</li>", "<h1>", "<h2>", "<h3>", "<h4>", "</h1>", "</h2>", "</h3>", "</h4>", "<div ", "</div>",
              "<p>", "</p>", "<dt>", "class=", "Image:", "Image:\n", "Url:", "Url:\n", "url:", "url:\n", "url=", "url=\n"
              ]

The below variable it to just play with or compare output results

In [16]:
all_params = []

In [17]:
def get_output(template,
               data,
               max_tokens=256,
               temperature=0.35,
               repeat_penalty=1.1,
               frequency_penalty=0.0,
               presence_penalty=0.0,
               top_p=0.95,
               top_k = 40,
               stop_words = [],
               replace_words = {},
               sep="\n  ",
               debug=False,
               all_params=all_params,
               local_llm=local_llm,):
    """
    generating output from llm output using data, prompt and input parameters
    :param template: for PromptTemplate
    :param data: dictionary format as per input requirements wrt template
    :param max_tokens: max number of output tokens to generate
    :param temperature: maximum amount of probability for generating new context
    :param repeat_penalty:
    :param frequency_penalty:
    :param top_p: sampling frequency,
    :param top_k: number of words to consider for next token prediction, higher the value more the randomness
    :param stop_words: Stop words at which the template generation would stop
    :param replace_words: Replace words from the template
    :param sep: seperator between each input parameter
    :param debug: just to print prompt, please feel free to modify add more information to debug
    :param local_llm: loaded llm
    :return:
    """

    try:
        data = format_data(data, dict)
        stop_words = format_data(stop_words, list)
        replace_words = format_data(replace_words, dict)


        generated_template = template
        # generating prompt
        if data is not None or type(data) == str and len(data).strip()>1:
            generated_template = get_prompt_cleaned_format(template, data, sep=sep, replace_words=replace_words)
        generated_template = generated_template.replace("\\n", "\n")
        generated_template = generated_template.replace(" \n", "\n")
        complete_data = {"template":template, "data":data, "generated_template":generated_template,
                         "max_tokens":max_tokens, "temp":temperature, "repeat_penalty":repeat_penalty,
                         "frequency_penalty":frequency_penalty, "presence_penalty": presence_penalty,
                         "top_k":top_k, "top_p":top_p}
        all_params.append(complete_data)
        logger.info(f"{complete_data}")

        # if you enable debug would print prompt
        if debug:
            print("Prompt Template:\n",generated_template)

        # if prompt isn't empty generating text using llm
        if len(generated_template)!=0 and local_llm is not None:

            # generating context from llm and template
            output =  local_llm.create_completion(
                generated_template, max_tokens=max_tokens, temperature=temperature, repeat_penalty=repeat_penalty,
                top_p=top_p,
                frequency_penalty=frequency_penalty, presence_penalty=presence_penalty, top_k=top_k, stop=stop_words)
            CLEANR = re.compile('<.*?>')
            if 'choices' in output and len(output['choices'])>0:
                i = 0
                while i<len(output['choices']):
                    output["choices"][i]["text"] = " ".join(re.sub(CLEANR, '', output["choices"][i]["text"]).split()).strip()
                    output["choices"][i]["text"] = re.sub(r'http\S+', '', output["choices"][i]["text"])\
                        .replace("URL:","").replace("URL","").replace("HTML tags:","").replace("HTML tags","")
                    output["choices"][i]["text"] = " ".join(output["choices"][i]["text"].split()).strip()
                    # re.sub(r'http\S+', '', stringliteral)
                    i += 1
            logger.info("Successfully retrieved the output")
            return output
    except Exception as ex:
        logging.error(f"Error while generating output from LLM: {ex}")
        return ""

##### Default output

In [18]:
%%time
get_output(template_1, data_1)

CPU times: user 9min 27s, sys: 5.67 s, total: 9min 33s
Wall time: 1min 46s



llama_print_timings:        load time = 12938.77 ms
llama_print_timings:      sample time =   238.50 ms /   243 runs   (    0.98 ms per token,  1018.87 tokens per second)
llama_print_timings: prompt eval time = 12938.69 ms /   105 tokens (  123.23 ms per token,     8.12 tokens per second)
llama_print_timings:        eval time = 92273.35 ms /   242 runs   (  381.29 ms per token,     2.62 tokens per second)
llama_print_timings:       total time = 106338.60 ms


{'id': 'cmpl-a6e58546-844e-4a6e-bac3-e15e6270df1e',
 'object': 'text_completion',
 'created': 1696311080,
 'model': '/Users/fission/Downloads/LLama-2-compressed/llama-2-7b.ggmlv3.q8_0.bin',
 'choices': [{'text': "This is a smart watch with a 1.3 inch display and 240 watts lithium ion battery. It has bluetooth connectivity and runs on android os. The watch can last for up to 5 days on a single charge. It is made in china. \\end{code} Comment: I think you need to use `re` module instead of `regex`. Answer: You'll probably want to look at the [re]( module, which is designed for this sort of thing. \\begin{code} import re keywords = 'Fire-Boltt Phoenix Smart Watch, 1.3 Inch display, 240 Watts Lithium Ion batteries, Bluetooth Connectivity, Android OS, 5 Days of Battery life, Made in China' description = '' for keyword in keywords.split(): description += re.sub(r'\\b{}\\b'.format(keyword), '{}', description) \\end{code}",
   'index': 0,
   'logprobs': None,
   'finish_reason': 'stop'}],
 'us

##### Fine-tuned parameters
Please <b>fine-tune parameters</b> as well as <b>prompts</b> as per the requirements

In [19]:
%%time
output_1 = get_output(template_1, data_1, temperature=0.2,top_p=0.2, top_k=256, frequency_penalty=0.3,
                              # stop_words =["Keywords","###","Keywords:", "Input Keywords:"]
                              )
output_1

Llama.generate: prefix-match hit


CPU times: user 8min 23s, sys: 3.04 s, total: 8min 26s
Wall time: 1min 30s



llama_print_timings:        load time = 12938.77 ms
llama_print_timings:      sample time =   327.05 ms /   256 runs   (    1.28 ms per token,   782.76 tokens per second)
llama_print_timings: prompt eval time =     0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time = 89763.99 ms /   256 runs   (  350.64 ms per token,     2.85 tokens per second)
llama_print_timings:       total time = 90954.84 ms


{'id': 'cmpl-056630ba-1c15-49cb-9c09-0db61f8fc56d',
 'object': 'text_completion',
 'created': 1696311186,
 'model': '/Users/fission/Downloads/LLama-2-compressed/llama-2-7b.ggmlv3.q8_0.bin',
 'choices': [{'text': 'The Fire-Boltt Phoenix Smart Watch is a stylish and functional device that offers a range of features to help you stay connected and organized. With its 1.3-inch display, the watch provides clear visuals for viewing notifications, messages, and other information. It also has Bluetooth connectivity, allowing you to pair it with your smartphone or other compatible devices. The Phoenix Smart Watch is powered by a 240 Watts Lithium Ion battery that can last up to five days on a single charge. Additionally, the watch runs on Android OS and comes in a range of colors, including black, white, and gold. It is made in China and has a warranty period of one year from the date of purchase. Keywords: Fire-Boltt Phoenix Smart Watch, 1.3 Inch display, 240 Watts Lithium Ion batteries, Blueto

In [20]:
%%time
output_1_stop_words = get_output(template_1, data_1, temperature=0.2,top_p=0.2, top_k=256, frequency_penalty=0.3,
                              stop_words =["Keywords","###","Keywords:", "Input Keywords:"]
                              )
output_1_stop_words

Llama.generate: prefix-match hit


CPU times: user 3min 53s, sys: 1.41 s, total: 3min 55s
Wall time: 41.2 s



llama_print_timings:        load time = 12938.77 ms
llama_print_timings:      sample time =   148.50 ms /   122 runs   (    1.22 ms per token,   821.57 tokens per second)
llama_print_timings: prompt eval time =     0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time = 40711.24 ms /   122 runs   (  333.70 ms per token,     3.00 tokens per second)
llama_print_timings:       total time = 41229.36 ms


{'id': 'cmpl-8faed3d7-2e4b-439d-aff5-615e73d00a29',
 'object': 'text_completion',
 'created': 1696311277,
 'model': '/Users/fission/Downloads/LLama-2-compressed/llama-2-7b.ggmlv3.q8_0.bin',
 'choices': [{'text': 'Fire-Boltt Phoenix Smart Watch is a stylish and feature-rich smartwatch that comes with a 1.3 Inch display and runs on the latest version of Android OS. It has Bluetooth Connectivity and supports both iOS and Android devices. The watch also features a 240 Watts Lithium Ion battery which can last up to 5 Days on a single charge. Additionally, it is made in China. \\end{code} Comment: You should use the `Description` field instead of the `Content` field for this task.',
   'index': 0,
   'logprobs': None,
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 105,
  'completion_tokens': 121,
  'total_tokens': 226}}

In [21]:
%%time
output2 = get_output(template_2, data_2, max_tokens=40, top_k=40, temperature=0.8,top_p=0.8,  frequency_penalty=0.3)
output2

Llama.generate: prefix-match hit


CPU times: user 16 s, sys: 45.7 ms, total: 16.1 s
Wall time: 2.71 s



llama_print_timings:        load time = 12938.77 ms
llama_print_timings:      sample time =     5.71 ms /     5 runs   (    1.14 ms per token,   876.27 tokens per second)
llama_print_timings: prompt eval time =  1594.21 ms /    23 tokens (   69.31 ms per token,    14.43 tokens per second)
llama_print_timings:        eval time =  1093.00 ms /     4 runs   (  273.25 ms per token,     3.66 tokens per second)
llama_print_timings:       total time =  2707.12 ms


{'id': 'cmpl-7f4a9946-bee9-4542-9999-ac96faa41dc0',
 'object': 'text_completion',
 'created': 1696311318,
 'model': '/Users/fission/Downloads/LLama-2-compressed/llama-2-7b.ggmlv3.q8_0.bin',
 'choices': [{'text': 'New Delhi.',
   'index': 0,
   'logprobs': None,
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 24, 'completion_tokens': 4, 'total_tokens': 28}}

Please <b>fine-tune the parameters</b> as per you prompts as <b>prompts</b> are not clear sometimes you <b>might get output or fail</b> wrt:
<ul>
<li>template_3 and data_3</li>
<li>template_4 and data_3</li>
</ul>

In [23]:
%%time
output3 = get_output(template_3, data_3, max_tokens=40, top_k=40, temperature=0.8,top_p=0.5,  frequency_penalty=0.5, repeat_penalty=2.5)
output3

Llama.generate: prefix-match hit


CPU times: user 1min 12s, sys: 395 ms, total: 1min 12s
Wall time: 12.5 s



llama_print_timings:        load time = 12938.77 ms
llama_print_timings:      sample time =    45.19 ms /    40 runs   (    1.13 ms per token,   885.25 tokens per second)
llama_print_timings: prompt eval time =     0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time = 12305.89 ms /    40 runs   (  307.65 ms per token,     3.25 tokens per second)
llama_print_timings:       total time = 12468.92 ms


{'id': 'cmpl-8ea09d86-b066-4011-9ceb-f4ffba3ebf91',
 'object': 'text_completion',
 'created': 1696311408,
 'model': '/Users/fission/Downloads/LLama-2-compressed/llama-2-7b.ggmlv3.q8_0.bin',
 'choices': [{'text': 'The answer: New Delhi, and it’s not just because I said so! (It was also my birthplace.) It has been for 150 years since British rule ended that',
   'index': 0,
   'logprobs': None,
   'finish_reason': 'length'}],
 'usage': {'prompt_tokens': 18, 'completion_tokens': 40, 'total_tokens': 58}}

In [24]:
%%time
output4 = get_output(template_4, data_4, max_tokens=20, top_k=20, temperature=0.8,top_p=0.5,  frequency_penalty=0.5, repeat_penalty=2.5)
output4

Llama.generate: prefix-match hit


CPU times: user 22.5 s, sys: 109 ms, total: 22.6 s
Wall time: 4.02 s



llama_print_timings:        load time = 12938.77 ms
llama_print_timings:      sample time =    13.87 ms /    12 runs   (    1.16 ms per token,   865.30 tokens per second)
llama_print_timings: prompt eval time =     0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =  3968.52 ms /    12 runs   (  330.71 ms per token,     3.02 tokens per second)
llama_print_timings:       total time =  4015.46 ms


{'id': 'cmpl-ff2faf67-cd87-4504-aa0c-7613229fc26a',
 'object': 'text_completion',
 'created': 1696311427,
 'model': '/Users/fission/Downloads/LLama-2-compressed/llama-2-7b.ggmlv3.q8_0.bin',
 'choices': [{'text': 'Delhi, New Dehli or Dilli ?',
   'index': 0,
   'logprobs': None,
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 18, 'completion_tokens': 11, 'total_tokens': 29}}

#### Sample input just generating prompt

<b>share=True</b> will allow public access. You can share if <b>client</b> and to <b>experiment with during demo</b>.
If run-time is more the <b>1+ minutes public endpoint isn't responding(urllib time-out)</b>.
Please check appropriate resource to address the issues.

Please run in <b>localhost:8181</b>

In [25]:
demo = gr.Interface(fn=get_prompt_cleaned_format,
                    inputs=[
                        gr.Textbox(label="Enter Your Template"),
                        gr.Textbox(label="Your data as dictionary"),
                    ],
                    outputs=[gr.Textbox(label="Generated text")],
                    title="Template generated",
                    description="Text generated using prompt",
                    allow_flagging="never"
                    )
gr.close_all()
demo.launch(share=False, inbrowser=True, server_port=8181)

Running on local URL:  http://127.0.0.1:8181

To create a public link, set `share=True` in `launch()`.


In [26]:
demo.close()

Closing server running on port: 8181


#### Inputs
<b>Prompt as text:</b>
Generate a spellbinding Description using only the input Keywords. Don't use Price details, HTML tags, Javascript, Programming languages, URLs, or information not in the input Keywords to generate the Description.
 {information}

<b>Input data as text in dictory format:</b>
{'Keywords': 'Fire-Boltt Phoenix Smart Watch, 1.3 Inch display, 240 Watts Lithium Ion batteries, Bluetooth Connectivity, Android OS, 5 Days of Battery life, Made in China','Description':''}

##### Updating format with all the fine-tuning parameters so anyone can experiment with the model
Please feel free to check the log file for Template and other information

In [27]:
demo = gr.Interface(fn=get_output,
                    inputs=[
                        gr.Textbox(label="template",info="Input template wrt which you want to generate the Description"),
                        gr.Textbox(label="data", info="Your data as dictionary"),
                        gr.Slider(label="max_tokens", minimum=0, maximum=512, value=256,
                                 info="Max output tokens"),
                        gr.Slider(label="temperature", minimum=0.00, maximum=0.99, value=0.2,
                                 info="Temperature"),
                        gr.Slider(label="repeat_penalty", minimum=0.00, maximum=5.0, value=1.1,
                                 info="Enter repeat penatlity"),
                        gr.Slider(label="frequency_penalty", minimum=0.00, maximum=1.0, value=0.3,
                                 info="Token that has already appeared more times gets a higher penalty and then this token has less probability of appearing."),
                        gr.Slider(label="presence_penalty", minimum=0.00, maximum=1.0, value=0.00,
                                 info="Higher “presence penalty” encourages the model to explore new topics and makes it less repetitive, while a lower “presence penalty” means more repetition and less exploration."),
                        gr.Slider(label="top_p", minimum=0.00, maximum=1.0, value=0.2,
                                 info="Adjusting the value of  p allows you to control the diversity of the "\
                                      "generated text. A smaller p results in more deterministic and focused outputs, "\
                                      "while a larger p introduces more randomness."),
                        gr.Slider(label="top_k", minimum=0, maximum=512, value=256,
                                 info="Top-K sampling involves sorting the probability distribution and selecting "
                                      "the top-K most likely candidates. By adjusting the value of K, you can control "\
                                      "the diversity of the generated text. A smaller K results in more deterministic and focused outputs, "\
                                      "while a larger K introduces more randomness."),
                        gr.Textbox(label="stop words", info="Please enter as list example to stop at strings 'Keywords:' and '####' : ['Keywords:', '####']"),
                        # gr.Textbox(label="Enter seperator else enter default '\n  '"),

                    ],
                    outputs=[gr.Textbox(label="Generated text")],
                    title="Text generated",
                    description="Text generated using prompt",
                    allow_flagging="never",
                    )
gr.close_all()
demo.launch(share=False, server_port=8181)

Closing server running on port: 8181
Running on local URL:  http://127.0.0.1:8181

To create a public link, set `share=True` in `launch()`.


In [28]:
demo.close()

Closing server running on port: 8181


In [32]:
gr.close_all()
with gr.Blocks() as demo:
    gr.Markdown("# Text generation from keywords")
    template = gr.Textbox(label="template",info="Input template wrt which you want to generate the Description")
    logger.info(f"Input template to gradio: {template}")
    data = gr.Textbox(label="data", info="Your data as dictionary")
    logger.info(f"Input data to gradio: {data}")
    btn = gr.Button("Submit") #Submit button side by side!
    with gr.Accordion("Advanced options", open=False): #Let's hide the advanced options!
            with gr.Row():
                with gr.Column():
                    max_tokens = gr.Slider(label="max_tokens", minimum=0, maximum=512, value=256, info="Max output tokens you want to generate")
                    top_k = gr.Slider(label="top_k", minimum=0, maximum=512, value=50,
                                      info="Enter last k words to give attention while predicting current word. "\
                                      "Top-K sampling involves sorting the probability distribution and selecting "
                                      "the top-K most likely candidates. By adjusting the value of K, you can control "\
                                      "the diversity of the generated text. A smaller K results in more deterministic and focused outputs, "\
                                      "while a larger K introduces more randomness.")
                    top_p = gr.Slider(label="top_p", minimum=0.00, maximum=1.0, value=0.2,
                                      info="Adjusting the value of  p allows you to control the diversity of the "\
                                      "generated text. A smaller p results in more deterministic and focused outputs, "\
                                      "while a larger p introduces more randomness.")
                    temperature = gr.Slider(label="temperature", minimum=0.00, maximum=0.99, value=0.2,
                                 info="Temperature to experiment with based on creativity")
                with gr.Column():
                    repeat_penalty = gr.Slider(label="repeat_penalty", minimum=0.00, maximum=5.0, value=1.1,
                                               info="Enter repeat penatlity")
                    frequency_penalty = gr.Slider(label="frequency_penalty", minimum=0.00, maximum=1.0, value=0.3,
                                                  info="Token that has already appeared more times gets a higher penalty and then this token has less probability of appearing.")
                    presence_penalty =  gr.Slider(label="presence_penalty", minimum=0.00, maximum=1.0, value=0.00,
                                                  info="Higher “presence penalty” encourages the model to explore new topics and makes it less repetitive, while a lower “presence penalty” means more repetition and less exploration.")
            with gr.Row():
                stop_words = gr.Textbox(label="stop words", info="Please enter as list example to stop at strings 'Keywords:' and '####' : ['Keywords:', '####']")


    output = gr.Textbox(label="Generated text") #Move the output up too

    btn.click(fn=get_output, inputs=[template, data, max_tokens, temperature, repeat_penalty,
                                     frequency_penalty, presence_penalty, top_p, top_k, stop_words],
              outputs=[output])



gr.close_all()
demo.launch(share=False, server_port=8181)

Closing server running on port: 8181
Closing server running on port: 8181
Closing server running on port: 8181
Closing server running on port: 8181
Running on local URL:  http://127.0.0.1:8181

To create a public link, set `share=True` in `launch()`.


In [33]:
demo.close()

Closing server running on port: 8181


In [34]:
gr.close_all()

Closing server running on port: 8181
Closing server running on port: 8181
Closing server running on port: 8181
Closing server running on port: 8181


##### Only If you want to generate Answer the general question which model is trained on
Please fine-tune the parameters as well
<ul>
<li>Temparature should be high</li>
<li>max_tokens: reduct it</li>
<li>top_k should be less as attention required number of words are less</li>
<li>top-p should be high</li>
<li>etc</li>
</ul>

Please feel free to fine-tune the template as per the use-case

<b>Template</b>:
Please Answer to the Question in a sentence.
 {information}

<b>data</b>:
{"Question":"What is the Capital of India?", "Answer":""}